# 03. BindingDB Preprocessing

In [1]:
import pandas as pd
import numpy as np

Download BindingDB and unzip it: https://www.bindingdb.org/rwd/bind/chemsearch/marvin/SDFdownload.jsp?download_file=/bind/downloads/BindingDB_All_202411_tsv.zip

In [2]:
PATH_TO_BDB = '../../mol-finder/data/raw/bindingdb/BindingDB_All.tsv'
bdb = pd.read_csv(PATH_TO_BDB, sep='\t', on_bad_lines='skip', low_memory=False, usecols=[
    'Ligand SMILES',
    'Ki (nM)',
    'IC50 (nM)',
    'Kd (nM)',
    'EC50 (nM)',
    'UniProt (SwissProt) Primary ID of Target Chain',
    'UniProt (TrEMBL) Primary ID of Target Chain',
    'BindingDB Target Chain Sequence',
    'Number of Protein Chains in Target (>1 implies a multichain complex)',
])

bdb = bdb[bdb['Number of Protein Chains in Target (>1 implies a multichain complex)'] == 1]

bdb = bdb.rename({
    'Ligand SMILES': 'smiles',
    'Ki (nM)' : 'ki',
    'IC50 (nM)': 'ic50',
    'Kd (nM)': 'kd',
    'EC50 (nM)': 'ec50',
    'UniProt (SwissProt) Primary ID of Target Chain': 'uniprot_swissprot',
    'UniProt (TrEMBL) Primary ID of Target Chain': 'uniprot_trembl',
    'BindingDB Target Chain Sequence': 'sequence',
},  axis=1) 
bdb

,smiles,ki,ic50,kd,ec50,Number of Protein Chains in Target (>1 implies a multichain complex),sequence,uniprot_swissprot,uniprot_trembl
0,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,0.24,NaN,NaN,NaN,1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,P03367,NaN
1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,0.25,NaN,NaN,NaN,1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,P03367,NaN
2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,0.41,NaN,NaN,NaN,1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,P03367,NaN
3,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,0.8,NaN,NaN,NaN,1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,P03367,NaN
4,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,0.99,NaN,NaN,NaN,1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,P03367,NaN
...,...,...,...,...,...,...,...,...,...
2913280,CC[C@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc(...,NaN,NaN,NaN,670,1,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,P37238,NaN
2913281,CC[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc...,NaN,NaN,NaN,40,1,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,P37238,NaN
2913282,CC(C)[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2...,NaN,NaN,NaN,230,1,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,P37238,NaN
2913283,COc1ccc(cc1)N(C)c1nc(C)nc2[nH]ccc12,NaN,2600,NaN,NaN,1,CVSASPSTLARLVSRSAMPAGSSTAWNTAFSPMARCQVTKTIGGGD...,NaN,Q862L2


In [3]:
def extract_sign(val):
    if isinstance(val, float):
        return None  # it's nan
    val = str(val)
    if not val:
        return None
    if val[0] in ["<", ">"]:
        return val[0]
    else:
        return '='

def extract_val(val):
    if isinstance(val, float):
        return None  # it's nan
    val = str(val)
    if not val:
        return None
    if val[0] in ["<", ">"]:
        return float(val[1:])
    else:
        return float(val)

In [4]:
bdb['ki_sign'] = bdb['ki'].map(extract_sign)
bdb['ki'] = bdb['ki'].map(extract_val)
bdb['ic50_sign'] = bdb['ic50'].map(extract_sign)
bdb['ic50'] = bdb['ic50'].map(extract_val)
bdb['kd_sign'] = bdb['kd'].map(extract_sign)
bdb['kd'] = bdb['kd'].map(extract_val)
bdb['ec50_sign'] = bdb['ec50'].map(extract_sign)
bdb['ec50'] = bdb['ec50'].map(extract_val)

In [5]:
bdb = bdb.dropna(subset=['uniprot_swissprot', 'uniprot_trembl'], how='all')

def get_uniprot_id(row):
    if pd.notna(row['uniprot_swissprot']):
        return row['uniprot_swissprot']
    else:
        return row['uniprot_trembl']
bdb['uniprot_id'] = bdb.apply(get_uniprot_id, axis=1)
bdb = bdb.drop(labels=['uniprot_swissprot', 'uniprot_trembl', 'Number of Protein Chains in Target (>1 implies a multichain complex)'], axis=1)
bdb['source'] = 'bdb'
bdb


/var/tmp/ipykernel_129203/2535334968.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdb['uniprot_id'] = bdb.apply(get_uniprot_id, axis=1)


,smiles,ki,ic50,kd,ec50,sequence,ki_sign,ic50_sign,kd_sign,ec50_sign,uniprot_id,source
0,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,0.24,NaN,NaN,NaN,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,=,None,None,None,P03367,bdb
1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,0.25,NaN,NaN,NaN,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,=,None,None,None,P03367,bdb
2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,0.41,NaN,NaN,NaN,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,=,None,None,None,P03367,bdb
3,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,0.80,NaN,NaN,NaN,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,=,None,None,None,P03367,bdb
4,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,0.99,NaN,NaN,NaN,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,=,None,None,None,P03367,bdb
...,...,...,...,...,...,...,...,...,...,...,...,...
2913280,CC[C@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc(...,NaN,NaN,NaN,670.0,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,None,None,None,=,P37238,bdb
2913281,CC[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc...,NaN,NaN,NaN,40.0,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,None,None,None,=,P37238,bdb
2913282,CC(C)[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2...,NaN,NaN,NaN,230.0,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,None,None,None,=,P37238,bdb
2913283,COc1ccc(cc1)N(C)c1nc(C)nc2[nH]ccc12,NaN,2600.0,NaN,NaN,CVSASPSTLARLVSRSAMPAGSSTAWNTAFSPMARCQVTKTIGGGD...,None,=,None,None,Q862L2,bdb


In [6]:
bdb.to_csv('bdb.csv', index=False)